# jQAssistant Demos
* Neo4j-Server starten
 * `mvn jqassistant:server`
* Browser öffnen
 * http://localhost:7474/browser/
* Drawer öffnen
* Labels durchklicken
 * `Commit`
 * `Class`
 * `:DECLARES`
 * jQAssistant Dokumentation: http://buschmais.github.io/jqassistant/doc/1.3.0/#_java_plugin
 

## Beispiel-Queries

### Setup
Cypher Extension für Jupyter laden

In [ ]:
%load_ext cypher

### Klasse mit den meisten Methoden auflisten

In [ ]:
%%cypher
MATCH 
  (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Typ, COUNT(m) as Methoden
ORDER BY Methoden DESC

### Statische, geschriebene Variablen

In [ ]:
%%cypher
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField

### Aggregation von Messergebnissen über fachliche Bereiche

In [ ]:
%%cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes
ORDER BY Types DESC

### Umfassende Aggregation von Messergebnissen über fachliche Bereiche

In [ ]:
%%cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change),
  (t)-[:HAS_MEASURE]->(co:Coverage)
OPTIONAL MATCH
  (t)-[:HAS_BUG]->(b:BugInstance)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes,
  AVG(co.ratio) as Coverage,
  COUNT(DISTINCT b) as Bugs,
  SUM(DISTINCT t.lastMethodLineNumber) as Lines
ORDER BY Coverage ASC, Bugs DESC